In [1]:
import os
import pickle
from collections import Counter

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

npu = np.unique
npi = np.intersect1d
npc = np.concatenate

from senset import HLCA, PUC_ct, DE_test_ct, SncGeneSets, get_top_markers
from senset.utils import construct_PU_results_table
from senset.plots import pairwise_overlap_heatmap

## 1. Load Senescence gene sets

The file containing all four marker sets is in `data/senescence_list.xlsx`.

These include the following

- GO:0090398 (https://academic.oup.com/bioinformatics/article/25/2/288/220714)
- Fridman (https://www.nature.com/articles/onc2008213)
- SenMayo (https://www.nature.com/articles/s41467-022-32552-1)
- CellAge (https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-01990-9)

For the GO set, genes with symbol equal to NA were removed. We do not
remove downregulated genes. If you choose to do so, this will affect
Fridman (where only genes with `'Regulation' == 'UP'` are kept) and CellAge (where
only genes with `'Senescence Effect' == 'Induces'` are kept).

In [ ]:
SNC = SncGeneSets(remove_downregulated=False)  # Load 4 existing lists
print(SNC)

In [ ]:
pairwise_overlap_heatmap(
    [getattr(SNC, gene_set) for gene_set in SNC.gene_sets],
    ticklabels=SNC.gene_sets,
)

## 2. Load the Human Lung Cell Atlas (HLCA)

Data can be downloaded from
https://data.humancellatlas.org/hca-bio-networks/lung/atlases/lung-v1-0.
We only use the `core` dataset.

By default, we remove patients with `age == NA` (only 1) and also remove
active or former smokers. A few donors have 'smoking_status' = NA, so we
removed these too.

In [ ]:
hlca = HLCA('../SenSet/data/HLCA.h5ad', remove_nan_age=True, remove_smokers=True)
adata = hlca.adata

In [ ]:
A1, A2 = 30, 50
print(f"""
    N. cells = {adata.shape[0]}
    N. genes = {adata.shape[1]}
    N. cell types = {np.unique(adata.obs['cell_type']).size}
    N. donors = {np.unique(adata.obs['donor_id']).size}
    Min age = {np.min(hlca.age)}
    Max age = {np.max(hlca.age)}
    N. donors (a < {A1}) yo = {len([v for v in hlca.donor_age if v[1] < A1])}
    N. donors ({A1} <= a < {A2}) yo = {len([v for v in hlca.donor_age if v[1] >= A1 and v[1] < A2])}
    N. donors (a >= {A2}) yo = {len([v for v in hlca.donor_age if v[1] >= A2])}
""")

## 3. PUc Learning

For each cell type in the atlas, we reduce to $k$ components via PCA and
use this lower dimensional representation as input to the PUc learner.
Here, we run PCA on known senescence markers only. The prior (fraction of
healthy cells in the older groups) is set to 0.9.

In [6]:
extension = "SNC-known-pca=10-prior=0.9-no-smoker"

In [ ]:
PUc_results = {}

for cell_type in tqdm(np.unique(adata.obs['cell_type'])):
    PUc_results[cell_type] = PUC_ct(
        adata, cell_type,
        age_thresh=(A1, A2),
        n_components=10,  # PCA components
        known_markers=SNC.union,  # set to None to use all genes for PUc
        prior=0.9,
    )

os.makedirs('dumps', exist_ok=True)
with open(f'dumps/PU-{extension}.pkl', 'wb') as f:
    pickle.dump(PUc_results, f)

In [7]:
# with open(f"dumps/PU-{extension}.pkl", "rb") as f:
#     PUc_results = pickle.load(f)

In [8]:
table_df = construct_PU_results_table(PUc_results)
table_df.to_csv(f'dumps/table-df-{extension}.csv')

## 4. DE Tests

For each cell type, we perform a ranksum test to determine differentially
expressed genes. Known markers are used only. Alternatively, one can test all
the genes in the atlas (even if PUc was ran on known markers only).

In [ ]:
test_results = {}
for cell_type, result in tqdm(PUc_results.items()):
    if result is None:
        continue
    test_results[cell_type] = DE_test_ct(
        adata, cell_type,
        PUc_results[cell_type],
        min_n_snc_cells=10,  # min n cells per DE test
        known_markers=SNC.union,  # set to None for DE on full transcriptome
    )

with open(f'dumps/DE-test-results-{extension}.pkl', 'wb') as f:
    pickle.dump(test_results, f)

In [11]:
# with open(f'dumps/DE-test-results-{extension}.pkl', 'rb') as f:
#     test_results = pickle.load(f)

## 5. SenSet Construction

Select the top differentially expressed genes. We take all genes with
$\text{FDR}<0.05$ and then select the ones showing enrichment in most cell types.

In [10]:
# Use K=0 to take all that are significant
de_results = get_top_markers(test_results, K=0, Q=0.05, use_statistic=True)

In [ ]:
MIN_CELL_TYPES = 6  # Adjust this number to select varying number of genes
counts = Counter(npc([val['all'] for val in de_results.values()]))
senset = np.asarray([v[0] for v in counts.most_common() if v[1] >= MIN_CELL_TYPES])
senset.sort()
print(f"Found {len(senset)} marker genes.")

### Dump SenSet into a txt file

In [44]:
with open(f"dumps/SenSet-{extension}.txt", "w") as f:
    f.write(senset[0])
    for gene in senset[1:]:
        f.write(f"\n{gene}")